Import agate, a Python-based data analysis library

In [86]:
import agate

Create agate Table of csv file from Nebraska registered voters data from 2000 to 2014

In [87]:
parties = agate.Table.from_csv('CleanPoliticalData.csv')

Check out column names & types from parties table

In [88]:
print(parties)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  County Name  | Text          |
|  FIPS Code    | Number        |
|  Total2014    | Number        |
|  Total2012    | Number        |
|  Total2010    | Number        |
|  Total2008    | Number        |
|  Total2006    | Number        |
|  Total2004    | Number        |
|  Total2002    | Number        |
|  Total2000    | Number        |
|  NP2014       | Number        |
|  NP2012       | Number        |
|  NP2010       | Number        |
|  NP2008       | Number        |
|  NP2006       | Number        |
|  NP2004       | Number        |
|  NP2002       | Number        |
|  NP2000       | Number        |
|  GOP2014      | Number        |
|  GOP2012      | Number        |
|  GOP2010      | Number        |
|  GOP2008      | Number        |
|  GOP2006      | Number        |
|  GOP2004      | Number        |
|  GOP2002      | Number        |
|  GOP2000      | Number        |
|  Dem2014    

Let's see some stats about Nebraska political party distributions and calculate when the independent party would overtake the Democratic Party, given the growth rates continue.

Grab the total row form the parties table

In [26]:
totalRow = parties.where(lambda row: row['County Name'] == 'Total State')
print(len(totalRow.rows))

1


Calculate the percent of each party in 2000

In [61]:
def calculateNP00(row):
    return row['NP2000'] / row['Total2000']
def calculateDem00(row):
    return row['Dem2000'] / row['Total2000']
def calculateGOP00(row):
    return row['GOP2000'] / row['Total2000']

In [62]:
distributions00 = totalRow.compute([
        ('nonparty_distribution2000', agate.Formula(agate.Number(), calculateNP00)),
        ('demparty_distribution2000', agate.Formula(agate.Number(), calculateDem00)),
        ('gopparty_distribution2000', agate.Formula(agate.Number(), calculateGOP00)),
    ])

View each party's percent in 2000

In [64]:
printDistribution00 = distributions00.select(['nonparty_distribution2000', 'demparty_distribution2000', 'gopparty_distribution2000'])
printDistribution00.print_table()

|----------------------------+---------------------------+----------------------------|
|  nonparty_distribution2000 | demparty_distribution2000 | gopparty_distribution2000  |
|----------------------------+---------------------------+----------------------------|
|       0.141066717532069... |      0.361535066258637... |      0.495389401382396...  |
|----------------------------+---------------------------+----------------------------|


Calculate the percent of each party in 2014

In [65]:
def calculateNP14(row):
    return row['NP2014'] / row['Total2014']
def calculateDem14(row):
    return row['Dem2014'] / row['Total2014']
def calculateGOP14(row):
    return row['GOP2014'] / row['Total2014']

In [66]:
distributions14 = totalRow.compute([
        ('nonparty_distribution2014', agate.Formula(agate.Number(), calculateNP14)),
        ('demparty_distribution2014', agate.Formula(agate.Number(), calculateDem14)),
        ('gopparty_distribution2014', agate.Formula(agate.Number(), calculateGOP14)),
    ])

View each party's percent in 2014

In [69]:
printDistribution14 = distributions14.select(['nonparty_distribution2014', 'demparty_distribution2014', 'gopparty_distribution2014'])
printDistribution14.print_table()

|----------------------------+---------------------------+----------------------------|
|  nonparty_distribution2014 | demparty_distribution2014 | gopparty_distribution2014  |
|----------------------------+---------------------------+----------------------------|
|       0.203584619101860... |      0.308787235511373... |      0.482693037865451...  |
|----------------------------+---------------------------+----------------------------|


Calculate how long until the independent party overtakes the Democratic party. Using the Investigative Reporters & Editors' Number in the Newsrooms, I took the average of the last three election years other than the last. Then, I divided the last year by that average. That percent change was taken to the third power, which computes the percent growth in three election cycles or 6 years. Finally, the last year is multiplied by this growth value to find the estimated party numbers 6 years from 2014 or 2020. 

In [79]:
def futureNP(row):
    average = (row['NP2012'] + row['NP2010'] + row['NP2008'])/3
    percentChange = row['NP2014'] / average
    change3elections = percentChange ** 3
    futureValue = row['NP2014'] * change3elections
    return futureValue
def futureDem(row):
    average = (row['Dem2012'] + row['Dem2010'] + row['Dem2008'])/3
    percentChange = row['Dem2014'] / average
    change3elections = percentChange ** 3
    futureValue = row['Dem2014'] * change3elections
    return futureValue
def futureGOP(row):
    average = (row['GOP2012'] + row['GOP2010'] + row['GOP2008'])/3
    percentChange = row['GOP2014'] / average
    change3elections = percentChange ** 3
    futureValue = row['GOP2014'] * change3elections
    return futureValue


In [83]:
future = totalRow.compute([
        ('futureNP', agate.Formula(agate.Number(), futureNP)),
        ('futureDem', agate.Formula(agate.Number(), futureDem)),
        ('futureGOP', agate.Formula(agate.Number(), futureGOP)),
    ])

Given the same growth/decline rates, the parties will have the following estimated registered voters in 2020.

In [85]:
printfuture = future.select(['futureNP', 'futureDem', 'futureGOP'])
printfuture.print_table()

|-----------------------+----------------------+-----------------------|
|              futureNP |            futureDem |            futureGOP  |
|-----------------------+----------------------+-----------------------|
|  339,788.401958366... | 297,350.726469258... | 583,439.493192371...  |
|-----------------------+----------------------+-----------------------|


Let's compute percent change from 2010 to 2014

In [17]:
changes0014 = parties.compute([
        ('total_change', agate.PercentChange('Total2000', 'Total2014')),
        ('nonparty_change', agate.PercentChange('NP2000', 'NP2014')),
        ('gopparty_change', agate.PercentChange('GOP2000', 'GOP2014')),
        ('demparty_change', agate.PercentChange('Dem2000', 'Dem2014')),
    ])

Lets round those values to one decimal place

In [18]:
from decimal import Decimal

def round_total_change(row):
    return row['total_change'].quantize(Decimal('0.1'))

def round_nonparty_change(row):
    return row['nonparty_change'].quantize(Decimal('0.1'))

def round_gopparty_change(row):
    return row['gopparty_change'].quantize(Decimal('0.1'))

def round_demparty_change(row):
    return row['demparty_change'].quantize(Decimal('0.1'))

rounded_changes = changes0014.compute([
    ('total_change_round', agate.Formula(agate.Number(), round_total_change)),
    ('nonparty_change_round', agate.Formula(agate.Number(), round_nonparty_change)),
    ('gopparty_change_round', agate.Formula(agate.Number(), round_gopparty_change)),
    ('demparty_change_round', agate.Formula(agate.Number(), round_demparty_change)),
])

Ugh it's ugly lets see only the fields we need

In [19]:
for_printing = rounded_changes.select(['County Name', 'total_change_round', 'nonparty_change_round', 'gopparty_change_round', 'demparty_change_round'])

Sort the table so largest growing to smallest

In [20]:
sorted_change = for_printing.order_by('nonparty_change_round', reverse=True)

Print top 50 highest growing populations

In [21]:
sorted_change.print_table()

|---------------+--------------------+-----------------------+-----------------------+------------------------|
|  County Name  | total_change_round | nonparty_change_round | gopparty_change_round | demparty_change_round  |
|---------------+--------------------+-----------------------+-----------------------+------------------------|
|  MCPHERSON    |               -7.0 |                 228.6 |                  -2.4 |                 -41.8  |
|  SARPY        |               42.6 |                 107.8 |                  34.9 |                  17.4  |
|  HOOKER       |               -8.4 |                  82.8 |                 -10.7 |                 -21.6  |
|  KIMBALL      |                7.2 |                  81.6 |                   6.6 |                 -23.7  |
|  THURSTON     |               10.8 |                  75.8 |                  -9.1 |                  12.2  |
|  HAYES        |               -6.7 |                  72.3 |                  -4.7 |                 -

In [23]:
sorted_change.to_csv('neb-county-political.csv')